In [28]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime
import streamlit as st
import numpy as np
from datetime import date
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
BigQuery_client = bigquery.Client(credentials=credentials)
# THIS FILE PERFORMS THE ETL OF THE DATA HOSTED IN BIG QUERY AND DOWNLOADS THE FINAL FILE IN THE DATA FOLDER

group3_st=['IN','MI','MD','MA','OH','PA','IL','NJ','FL','TX','CA','GA']
group1_st=['WA','CO','NC','MN','NY','OR','AZ','VA']
group2_st=['SC','TN','UT','NV','AK','MO','LA','KY','DC','NE']
group4_st=['OK','RI','KS','CT','WI']

hoy=datetime.today()

def state_group(state):
    if state in group1_st:
        return "1"
    elif state in group2_st:
        return "2"
    elif state in group3_st:
        return "3"
    elif state in group4_st:
        return "4"
    else:
        return "Other"

group0=['Aaron Domke', 'Alex Aguilar', 'Andrew Rogers', 'Andrew Stein','Christiana Merdaa', 'Daniel Lombardo', 'David Gulakowski','Debra Morris', 'Demery Moody', 'Eran Even-Kesef', 'Gary Schwartz','James Loomis', 'Jason Chew', 'Jason Mankevich', 'Ladeen Mccray-Davis','Laura Rowe', 'Lucy Shockney', 'Nicholas Wilsey', 'Patrick Markowski','Paul Kanengiser', 'Ricky Rodemacher', 'Robert Wolfe', 'Ryan Weaver','Shannon Rhodes', 'Shay Bar Nissim', 'Stephen Blanchfill', 'Troy C-Rep','Vanessa Berrueta', 'Vicky Barnett', 'Whitney Ross']
group1=['Amy Teague', 'Brian Hunter', 'Brian Sundberg', 'Brien Hebb','Carol Monaco', 'Crystal Benner', 'Dawn Devereux', 'Eric Moskow','Farah Elturk', 'Fernando Andrews', 'Gene Spinner', 'Jim Ryan','Kim Stotsky', 'Ray Patterson', 'Rudi Lovell', 'Warren Sabo']
group2=['Elizabeth Reynoso', 'Judd Stewart', 'Lisa Buzonik']
group3=['Brent Hicks', 'Chris Charbonneau', 'Jim Harness', 'Joseph Bonham']
group4=['Alan Veronese', 'Anthony Canton', 'Brett Buckingham','Christi Finnell', 'Dawn Wagner', 'Jon Dowling', 'Mark Howard','Misty Boodt', 'Nancy Braier', 'Nicole Hernandez', 'Robin Harp','Robin Reinhold', 'Scott Watkins', 'Tom Leverone']
group5=['Sammie Harris']
group6=['Alex Atanassov', 'Karen Embly']
def rep_cluster(name):
    if name in group0:
        return "0"
    elif name in group1:
        return "1"
    elif name in group2:
        return "2"
    elif name in group3:
        return "3"
    elif name in group4:
        return "4"
    elif name in group5:
        return "5"
    elif name in group6:
        return "6"
    else:
        return "Other"

query="""
WITH avg_revisit_visit AS
(
    SELECT account_id as id, AVG(DATE_DIFF(DATE(offer_date), DATE(PriorDate),MONTH)) as avg_revisit_visit, CAST(MIN(offer_date) as DATE) as first_visit_date,CAST(MAX(offer_date) as DATE) as last_visit_date
    FROM (
    SELECT
        account_id,
        offer_date,
        status,
        LAG(offer_date) OVER (PARTITION BY account_id ORDER BY offer_date) as PriorDate
    FROM `star-big-data.star_us_rds.offers`
    WHERE EXTRACT(YEAR FROM offer_date) > 2018)
    WHERE status <> 'NoVisit'
    GROUP BY account_id
    ORDER BY account_id 
),
avg_rebuy_months AS
(
    SELECT account_id as id, AVG(DATE_DIFF(DATE(offer_date), DATE(PriorDate),MONTH)) as avg_rebuy_months
    FROM (
    SELECT
        account_id,
        offer_date,
        LAG(offer_date) OVER (PARTITION BY account_id ORDER BY offer_date) as PriorDate
    FROM `star-big-data.star_us_rds.offers`
    WHERE EXTRACT(YEAR FROM offer_date) > 2018 AND status='Accepted') 
    GROUP BY account_id
    ORDER BY account_id
),
acc_age AS
(
    SELECT account_id as id,DATE_DIFF(DATE(CURRENT_DATE()), DATE(first_order_date),MONTH) as acc_age
    FROM(
    select account_id, MIN(offer_date) as first_order_date
    from `star-big-data.star_us_rds.offers`
    where EXTRACT(YEAR FROM offer_date) > 2018
    group by account_id
    order by account_id)
),
total_spent AS
(
    SELECT account_id as id, IFNULL(SUM(total_offer), 0 ) as total_spent
    FROM `star-big-data.star_us_rds.offers`
    where EXTRACT(YEAR FROM offer_date) > 2018 AND status='Accepted'
    group by account_id
    order by account_id
),
accounts AS
(
    SELECT c.id,c.specialty,c.accountstatus,c.phone,c.territory_id,c.last_visit_outcome,c.lastoffer_id,c.zip_left,c.last_offer_date,c.last_accepted_date,c.place_id,t.name as territory_name,c.accountrating,c.rep_id,c.practice_name,c.last_visit_date,p.city,p.longitude,p.latitude,p.county,p.state,r.name,r.inactive,
    CASE
        WHEN c.accountstatus = 0 then 'Lead'
        WHEN c.accountstatus = 1 then 'Prospect'
        WHEN c.accountstatus = 2 then 'Opportunity'
        WHEN c.accountstatus = 3 then 'Lost'
        WHEN c.accountstatus = 4 then 'Deleted'
        WHEN c.accountstatus = 5 then 'NPS'
    END as status_txt,
    CASE
        WHEN c.accountrating = 0 then 'Best'
        WHEN c.accountrating = 1 then 'Good'
        WHEN c.accountrating = 2 then 'Fair'
        WHEN c.accountrating = 3 then 'Poor'
        WHEN c.accountrating = 4 then 'Unqualified'
    END as rating_txt,
    FROM `star-big-data.star_us_rds.accounts` c
    LEFT JOIN `star-big-data.star_us_rds.places` p ON c.place_id=p.id 
    LEFT JOIN `star-big-data.star_us_rds.reps` r ON c.rep_id = r.id
    LEFT JOIN `star-big-data.star_us_rds.Territory` t ON c.territory_id = t.id
    WHERE c.accountstatus IS NOT NULL and c.accountrating IS NOT NULL
),offers AS
(
    SELECT o.id as offer_id, o.longitude,o.latitude,o.outcome_details,o.status,o.rep_id,r.name,r.inactive,o.total_offer,o.account_id,CAST(o.offer_date as DATE) as offer_date,  o.territory_id,t.name as territory_name,
    CASE
        WHEN status='Accepted' then 1
        WHEN status<>'Accepted' then 0
    END as buy,1 as visit , CAST(lag(o.offer_date) over (partition by o.account_id order by o.offer_date) as DATE) AS previous_visit_date
    FROM `star-big-data.star_us_rds.offers` o
    LEFT JOIN `star-big-data.star_us_rds.reps` r
    ON o.rep_id = r.id
    LEFT JOIN `star-big-data.star_us_rds.Territory` t
    ON o.territory_id = t.id
    WHERE EXTRACT(YEAR FROM offer_date) > 2018 and o.offer_date IS NOT NULL and o.account_id IS NOT NULL and r.name <> 'General Pot' and r.name <> 'Nancy Braier' and o.status <> 'NoVisit'
    ORDER BY o.offer_date ASC
),cummulative AS
(
    SELECT offer_id ,account_id,SUM(visit) OVER (PARTITION BY account_id ORDER BY offer_id) - 1 as cum_visits, SUM(buy) OVER (PARTITION BY account_id ORDER BY offer_id) - buy as cum_buys
    FROM offers     
    WHERE account_id IS NOT NULL
),cummulative_data AS
(
    SELECT account_id, max(cum_visits) as cum_prevvisits,max(cum_buys) as cum_prevbuys
    FROM cummulative
    GROUP BY account_id
),
acc_statusrating as
(
    select id , CONCAT(status_txt,' ',rating_txt) as statusandrating
    FROM accounts
)
SELECT a.id, a.specialty, a.accountstatus, a.phone, a.territory_id, a.last_visit_outcome, a.lastoffer_id,a.zip_left,CAST(a.last_offer_date as DATE) as last_offer_date,CAST(a.last_accepted_date as DATE) as last_accepted_date,a.place_id, a.territory_name,a.accountrating,a.rep_id,a.practice_name,CAST(a.last_visit_date as DATE) as last_visit_date,a.city,a.longitude,a.latitude,a.county,a.state,a.name,a.inactive,ts.total_spent,aa.acc_age,arm.avg_rebuy_months,arv.avg_revisit_visit, arv.first_visit_date,arv.last_visit_date as last_visit_prueba, DATE_DIFF(DATE(CURRENT_DATE()), DATE(arv.first_visit_date),MONTH) as months_between_firstvisit,
CASE
    WHEN DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_accepted_date),MONTH) < 24 then DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_accepted_date),MONTH)
    WHEN DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_accepted_date),MONTH) >= 24 then NULL
END as months_since_last_offer, 
CASE
    WHEN DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_visit_date),MONTH) < 24 then DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_visit_date),MONTH) 
    WHEN DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_visit_date),MONTH) >= 24 then NULL 
END as months_since_last_visit, asr.statusandrating, cd.cum_prevvisits, cd.cum_prevbuys,
CASE
    WHEN (cum_prevbuys=0 or cum_prevbuys IS NULL) or DATE_DIFF(DATE(CURRENT_DATE()), DATE(a.last_visit_date),MONTH) >= 24 then 1
    ELSE 0
END as prospect
FROM accounts a
LEFT JOIN acc_age aa ON aa.id=a.id
LEFT JOIN avg_rebuy_months arm ON arm.id=a.id
LEFT JOIN avg_revisit_visit arv ON arv.id=a.id
LEFT JOIN total_spent ts ON a.id=ts.id
LEFT JOIN acc_statusrating asr ON asr.id=a.id
LEFT JOIN cummulative_data cd ON cd.account_id = a.id
WHERE( a.specialty<>'Private Buys' or a.specialty IS NULL)  and statusandrating <> 'Deleted Poor' and statusandrating <> 'NPS Poor' and statusandrating <> 'Lead Unqualified' and statusandrating <> 'Lost Poor' 
"""


def cargar_datos():
    df = (BigQuery_client.query(query).result().to_dataframe(create_bqstorage_client=True,))
    df['state_cluster']=df['state'].apply(state_group)
    df['rep_cluster']=df["name"].apply(rep_cluster)
    return df


In [73]:
df = cargar_datos()

In [74]:
day = date.today()

In [75]:
df['months_between_firstvisit']=(day-pd.to_datetime(df['first_visit_date']).dt.date)/np.timedelta64(1, 'M')
df['months_between_previousvisit']=(day-pd.to_datetime(df['last_visit_date']).dt.date)/np.timedelta64(1, 'M')

In [76]:
type(df.loc[df['id']==488129,['last_visit_date','last_visit_prueba','first_visit_date']]['last_visit_date'].values[0])

NoneType

In [88]:
df['iguales']=(df['last_visit_date']==df['last_visit_prueba']).astype(int)

In [97]:
df.loc[(df['iguales']==0)&(~df['last_visit_date'].isna()),['id','last_visit_date','last_visit_prueba']]

,id,last_visit_date,last_visit_prueba
2,545505,2021-09-29,None
16,13995,2022-02-11,2019-07-10
18,541069,2022-02-07,2021-09-17
19,155571,2016-02-02,None
38,494543,2021-09-27,None
...,...,...,...
73667,442209,2020-11-04,2020-07-13
73680,546705,2022-02-10,None
73687,90787,2016-03-22,None
73691,494339,2022-04-18,None


In [94]:
ofertas_query="""
            SELECT o.id, o.longitude,o.latitude,o.outcome_details,o.status,o.rep_id,r.name,r.inactive,o.total_offer,o.account_id,o.offer_date,  o.territory_id,t.name as territory_name
            FROM `star-big-data.star_us_rds.offers` o
            LEFT JOIN `star-big-data.star_us_rds.reps` r
            ON o.rep_id = r.id
            LEFT JOIN `star-big-data.star_us_rds.Territory` t
            ON o.territory_id = t.id
    """

offers = (BigQuery_client.query(ofertas_query).result().to_dataframe(create_bqstorage_client=True,))

In [121]:
accounts_query="""
        SELECT c.id,c.specialty,c.practice_name,c.accountstatus,c.phone,c.territory_id,c.last_visit_outcome,c.lastoffer_id,c.zip_left,c.last_offer_date,c.last_accepted_date,c.place_id,t.name as territory_name,c.accountrating,c.practice_name,c.last_visit_date,p.city,p.longitude ,p.latitude ,p.county,p.state,c.rep_id,r.name,r.inactive
        FROM `star-big-data.star_us_rds.accounts` c
        LEFT JOIN `star-big-data.star_us_rds.places` p 
        ON c.place_id=p.id 
        LEFT JOIN `star-big-data.star_us_rds.Territory` t
        ON c.territory_id = t.id
        LEFT JOIN `star-big-data.star_us_rds.reps` r
        ON c.rep_id = r.id
    """
accounts = (BigQuery_client.query(accounts_query).result().to_dataframe(create_bqstorage_client=True,))

In [114]:
offers[offers['account_id'].isin([494339,546705,494543])]

,id,longitude,latitude,outcome_details,status,rep_id,name,inactive,total_offer,account_id,offer_date,territory_id,territory_name


In [122]:
offers[offers['practice_name']=='Panther Hollow Dental Lodge']

KeyError: 'practice_name'

In [119]:
accounts.loc[accounts['id'].isin([494339,546705,494543]),['id','last_visit_date','practice_name','name']]

,id,last_visit_date,practice_name,name
9705,494543,2021-09-27 17:50:41+00:00,Dhiren Mistry DDS,Joseph Bonham
112432,546705,2022-02-10 18:29:41+00:00,Panther Hollow Dental Lodge,Alex Aguilar
115483,494339,2022-04-18 14:49:07+00:00,Leading Edge Dental Center,Joseph Bonham


In [30]:
dataframe.loc[dataframe['id']==488053,['id','cum_prevvisits', 'months_between_firstvisit','months_between_previousvisit', 'avg_revisit_visit','rep_cluster', 'state_cluster']]

,id,cum_prevvisits,months_between_firstvisit,months_between_previousvisit,avg_revisit_visit,rep_cluster,state_cluster
26117,488053.0,NaN,NaN,NaN,4.333333,0,3


In [31]:
status={0:"Lead",1:"Prospect",2:"Opportunity",3:"Lost",4:"Deleted",5:"NPS"}
rating={ 0:"Best", 1:"Good",2:"Fair",3:"Poor",4:"Unqualified"}
def cargar_datos1():
    accounts_query="""
        SELECT c.id,c.specialty,c.accountstatus,c.phone,c.territory_id,c.last_visit_outcome,c.lastoffer_id,c.zip_left,c.last_offer_date,c.last_accepted_date,c.place_id,t.name as territory_name,c.accountrating,c.practice_name,c.last_visit_date,p.city,p.longitude ,p.latitude ,p.county,p.state,c.rep_id,r.name,r.inactive
        FROM `star-big-data.star_us_rds.accounts` c
        LEFT JOIN `star-big-data.star_us_rds.places` p 
        ON c.place_id=p.id 
        LEFT JOIN `star-big-data.star_us_rds.Territory` t
        ON c.territory_id = t.id
        LEFT JOIN `star-big-data.star_us_rds.reps` r
        ON c.rep_id = r.id
    """

    ofertas_query="""
            SELECT o.id, o.longitude,o.latitude,o.outcome_details,o.status,o.rep_id,r.name,r.inactive,o.total_offer,o.account_id,o.offer_date,  o.territory_id,t.name as territory_name
            FROM `star-big-data.star_us_rds.offers` o
            LEFT JOIN `star-big-data.star_us_rds.reps` r
            ON o.rep_id = r.id
            LEFT JOIN `star-big-data.star_us_rds.Territory` t
            ON o.territory_id = t.id
            where EXTRACT(YEAR FROM offer_date) > 2018
    """


    accounts = (BigQuery_client.query(accounts_query).result().to_dataframe(create_bqstorage_client=True,))
    offers = (BigQuery_client.query(ofertas_query).result().to_dataframe(create_bqstorage_client=True,))

    offers['offer_date']=pd.to_datetime(offers['offer_date'],errors='coerce')

    offers = offers[(offers['offer_date'].notna())&(offers['account_id'].notna())].sort_values('offer_date')
    offers=offers[~offers['name'].isin(['General Pot','Nancy Braier'])]
    offers=offers[offers['status']!='NoVisit']

    offers['buy']=(offers['status']=='Accepted').astype(int)
    offers['visit']=1

    offers=offers.merge(offers.groupby(['account_id'])['offer_date'].min().reset_index().rename(columns={'offer_date':'first_visit_date'}),how='left',on='account_id')
    offers=offers.merge(offers.groupby(['account_id'])['offer_date'].max().reset_index().rename(columns={'offer_date':'last_visit_date'}),how='left',on='account_id')

    offers['previous_visit_date'] = offers.groupby(['account_id'])['offer_date'].apply(lambda x: x.shift())
    offers['offer_date']=pd.to_datetime(offers['offer_date']).dt.date
    offers['first_visit_date']=pd.to_datetime(offers['first_visit_date']).dt.date
    offers['previous_visit_date']=pd.to_datetime(offers['previous_visit_date']).dt.date

    offers.loc[(((offers['offer_date']-offers['previous_visit_date'])/np.timedelta64(1, 'M'))>=24),'first_visit_date']=offers.loc[(((offers['offer_date']-offers['previous_visit_date'])/np.timedelta64(1, 'M'))>=24),'offer_date']
    offers.loc[(((offers['previous_visit_date']-offers['first_visit_date'])/np.timedelta64(1, 'M'))<0),'previous_visit_date']=np.nan

    offers=offers[offers['offer_date'] >= offers.groupby('account_id')['first_visit_date'].transform('max') ].sort_values('offer_date')

    offers=offers[(pd.to_datetime(offers['first_visit_date']).dt.year>=2018)]
    offers['cum_prevbuys'] = offers.groupby(['account_id'])['buy'].apply(lambda x: x.shift().cumsum()).fillna(0)
    offers['cum_prevvisits'] = offers.groupby(['account_id'])['visit'].apply(lambda x: x.shift().cumsum()).fillna(0)
    offers['months_between_firstvisit']=(offers['offer_date']-offers['first_visit_date']).dt.days/60
    offers['months_between_previousvisit']=(offers['offer_date']-offers['previous_visit_date']).dt.days/60
    offers['avg_months_visits']=offers['months_between_firstvisit']/offers['cum_prevvisits']

    temp=offers[['account_id','offer_date','avg_months_visits','cum_prevvisits','cum_prevbuys','first_visit_date']].sort_values('offer_date').groupby('account_id').tail(1)[['account_id','avg_months_visits','cum_prevvisits','cum_prevbuys','first_visit_date']]

    df_new=accounts.merge(temp,how='left',left_on='id',right_on='account_id')
    df_new['first_visit_date']=df_new['first_visit_date'].fillna(hoy)
    df_new['state_cluster']=df_new['state'].apply(state_group)
    df_new['rep_cluster']=df_new["name"].apply(rep_cluster)
    df_new['cum_prevbuys']=df_new['cum_prevbuys'].fillna(0)
    df_new=df_new[df_new['cum_prevbuys']==0]

    df_new['accountstatus_txt']=df_new['accountstatus'].replace(status)
    df_new['accountrating_txt']=df_new['accountrating'].replace(rating)
    df_new['statusandrating']=df_new['accountstatus_txt']+' '+df_new['accountrating_txt']
    df_new=df_new[~df_new['accountstatus'].isin([np.nan])]
    df_new=df_new[~df_new['accountrating'].isin([np.nan])]
    df_new=df_new[~df_new['statusandrating'].isin(['Deleted Poor','NPS Poor','Lead Unqualified','Lost Poor'])]


    avg_revisit_visit= """
        SELECT account_id, AVG(DATE_DIFF(DATE(offer_date), DATE(PriorDate),MONTH)) as avg_revisit_visit
        FROM (
        SELECT
            account_id,
            offer_date,
            LAG(offer_date) OVER (PARTITION BY account_id ORDER BY offer_date) as PriorDate
        FROM `star-big-data.star_us_rds.offers`
        WHERE EXTRACT(YEAR FROM offer_date) > 2018)
        GROUP BY account_id
        ORDER BY account_id
        """
    avg_rebuy_months= """
            SELECT account_id, AVG(DATE_DIFF(DATE(offer_date), DATE(PriorDate),MONTH)) as avg_rebuy_months
            FROM (
            SELECT
                account_id,
                offer_date,
                LAG(offer_date) OVER (PARTITION BY account_id ORDER BY offer_date) as PriorDate
            FROM `star-big-data.star_us_rds.offers`
            WHERE EXTRACT(YEAR FROM offer_date) > 2018 AND status='Accepted') 
            GROUP BY account_id
            ORDER BY account_id
            """
    acc_age="""
            SELECT account_id,DATE_DIFF(DATE(CURRENT_DATE()), DATE(first_order_date),MONTH) as acc_age
            FROM(
            select account_id, MIN(offer_date) as first_order_date
            from `star-big-data.star_us_rds.offers`
            where EXTRACT(YEAR FROM offer_date) > 2018
            group by account_id
            order by account_id)
    """
    total_spent="""
            SELECT account_id, IFNULL(SUM(total_offer), 0 ) as total_spent
            FROM `star-big-data.star_us_rds.offers`
            where EXTRACT(YEAR FROM offer_date) > 2018 AND status='Accepted'
            group by account_id
            order by account_id
    """
    accounts="""
            SELECT c.id,c.specialty,c.accountstatus,c.phone,c.territory_id,c.last_visit_outcome,c.lastoffer_id,c.zip_left,c.last_offer_date,c.last_accepted_date,c.place_id,t.name as territory_name,c.accountrating,c.rep_id,c.practice_name,c.last_visit_date,p.city,p.longitude,p.latitude,p.county,p.state,r.name,r.inactive
            FROM `star-big-data.star_us_rds.accounts` c
            LEFT JOIN `star-big-data.star_us_rds.places` p 
            ON c.place_id=p.id 
            LEFT JOIN `star-big-data.star_us_rds.reps` r
            ON c.rep_id = r.id
            LEFT JOIN `star-big-data.star_us_rds.Territory` t
            ON c.territory_id = t.id
    """


    dataframe1 = (BigQuery_client.query(total_spent).result().to_dataframe(create_bqstorage_client=True,))
    dataframe2 = (BigQuery_client.query(acc_age).result().to_dataframe(create_bqstorage_client=True,))
    dataframe3 = (BigQuery_client.query(avg_rebuy_months).result().to_dataframe(create_bqstorage_client=True,))
    dataframe4 = (BigQuery_client.query(avg_revisit_visit).result().to_dataframe(create_bqstorage_client=True,))
    dataframe5 = (BigQuery_client.query(accounts).result().to_dataframe(create_bqstorage_client=True,))

    df = dataframe1.merge(dataframe2,how='left',on='account_id').merge(dataframe3,how='left',on='account_id').merge(dataframe4,how='left',on='account_id').rename(columns={'account_id':'id'})

    dataframe5['id']=dataframe5['id'].astype(float)

    dataframe=dataframe5.merge(df,how='left',on='id').sort_values('id')
    dataframe['total_spent']=dataframe['total_spent'].fillna(0)
    dataframe['state_cluster']=dataframe['state'].apply(state_group)
    dataframe['rep_cluster']=dataframe["name"].apply(rep_cluster)
    dataframe[['acc_age','avg_rebuy_months','avg_revisit_visit']]=dataframe[['acc_age','avg_rebuy_months','avg_revisit_visit']].fillna(dataframe[['acc_age','avg_rebuy_months','avg_revisit_visit']].quantile(0.5))

    dataframe['last_accepted_date']=pd.to_datetime(dataframe['last_accepted_date']).dt.date
    dataframe['last_visit_date']=pd.to_datetime(dataframe['last_visit_date']).dt.date
    dataframe[['last_accepted_date','last_visit_date']]=dataframe[['last_accepted_date','last_visit_date']].fillna('')
    dataframe['months_since_last_offer']=(hoy-pd.to_datetime(dataframe['last_accepted_date']))/np.timedelta64(1, 'M')
    dataframe['months_since_last_visit']=(hoy-pd.to_datetime(dataframe['last_visit_date']))/np.timedelta64(1, 'M')
    dataframe['months_since_last_offer']=dataframe['months_since_last_offer'].apply(lambda x: x if x<24 else np.nan)
    dataframe['months_since_last_visit']=dataframe['months_since_last_visit'].apply(lambda x: x if x<24 else np.nan)

    dataframe['accountstatus_txt']=dataframe['accountstatus'].replace(status)
    dataframe['accountrating_txt']=dataframe['accountrating'].replace(rating)
    dataframe['statusandrating']=dataframe['accountstatus_txt']+' '+dataframe['accountrating_txt']
    dataframe=dataframe[~dataframe['accountstatus'].isin([np.nan])]
    dataframe=dataframe[~dataframe['accountrating'].isin([np.nan])]
    dataframe=dataframe[~dataframe['statusandrating'].isin(['Deleted Poor','NPS Poor','Lead Unqualified','Lost Poor'])]
    df_new=df_new[df_new['specialty']!='Private Buys']
    dataframe=dataframe[dataframe['specialty']!='Private Buys']
    return df_new,dataframe

In [32]:
dataframe_new,dataframe_old=cargar_datos1()

In [27]:
dataframe_new['last_visit_date']=pd.to_datetime(dataframe_new['last_visit_date']).dt.date
ids_nuevos=dataframe_new['account_id'].unique().tolist()
dataframe_old=dataframe_old[~dataframe_old['id'].isin(ids_nuevos)]
list1=set(dataframe_old.columns)-set(dataframe_old.columns)
list2=set(dataframe_new.columns)-set(dataframe_new.columns)
dataframe_new[list(list1)]=np.nan
dataframe_old[list(list2)]=np.nan
dataframe_new['prospect']=1
dataframe_old['prospect']=0
dataframe=dataframe_new.append(dataframe_old)
dataframe['months_between_firstvisit']=(day-pd.to_datetime(dataframe['first_visit_date']).dt.date)/np.timedelta64(1, 'M')
dataframe['months_between_previousvisit']=(day-pd.to_datetime(dataframe['last_visit_date']).dt.date)/np.timedelta64(1, 'M')

/var/folders/ls/r3k1n_891_nf47gq9lchwwr40000gn/T/ipykernel_9087/603922836.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe_new.append(dataframe_old)


In [29]:
dataframe.loc[dataframe['id']==488053,['id','cum_prevvisits', 'months_between_firstvisit','months_between_previousvisit', 'avg_revisit_visit','rep_cluster', 'state_cluster']]

,id,cum_prevvisits,months_between_firstvisit,months_between_previousvisit,avg_revisit_visit,rep_cluster,state_cluster
26117,488053.0,NaN,NaN,NaN,4.333333,0,3


## Casas de los reps

In [86]:
import requests
import urllib.parse
import numpy as np
import json
import pandas

In [79]:
def get_latlon(address):
    try:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
        response = requests.get(url).json()
        lat=response[0]["lat"]
        lon=response[0]["lon"]
    except Exception as e:
        lat,lon=np.nan,np.nan
    return pd.Series([lat,lon])

In [85]:
reps="""
SELECT id,name,email,address
FROM `star-big-data.star_us_rds.reps`
WHERE inactive = False and name <> 'General Pot'
"""
rep = (BigQuery_client.query(reps).result().to_dataframe(create_bqstorage_client=True,))
rep.dropna(inplace=True)
rep['address']=rep['address'].apply(json.loads)
rep['address']=rep['address'].apply(lambda x: x['address']+', '+x['city']+', '+x['state']+', '+x['country'])
rep[['lat','lon']]=rep['address'].apply(get_latlon)
rep.head()

,id,name,email,address,lat,lon
0,319,Michael Beller,mbeller@starrefining.com,"1441 E. Germann Rd. Apt 2021, Chandler, AZ, USA",NaN,NaN
1,317,Joseph Bonham,jbonham@starrefining.com,"210 South Finley Road, Lombard, IL, USA",41.88326814285714,-88.02986071428572
2,320,Robert Gregg,rgregg@starrefining.com,"3088 Blackberry Ln., Malvern, PA, USA",NaN,NaN
3,253,Eran Even-Kesef,eevenkesef@starrefining.com,"2655 Henley Pkwy, Unit 7511, Tracy, CA, USA",NaN,NaN
4,268,Troy C-Rep,tcrawford@starrefining.com,"1260 Chatham Road, Waynesboro, VA, USA",38.06102038952619,-78.91993873253134


In [90]:
rep[['id','name','address','lat','lon']].to_excel('repHouses.xlsx')

In [92]:
pd.read_excel('repHouses.xlsx')

,id,name,address,lat,lon
0,319,Michael Beller,"1441 E. Germann Rd. Apt 2021, Chandler, AZ, USA",33.277010,-111.814860
1,317,Joseph Bonham,"210 South Finley Road, Lombard, IL, USA",41.883268,-88.029861
2,320,Robert Gregg,"3088 Blackberry Ln., Malvern, PA, USA",40.072200,-75.561970
3,253,Eran Even-Kesef,"2655 Henley Pkwy, Unit 7511, Tracy, CA, USA",37.471320,-121.153610
4,268,Troy C-Rep,"1260 Chatham Road, Waynesboro, VA, USA",38.061020,-78.919939
5,315,Paul Kanengiser,"154 Hillside Ave, Springfield, NJ, USA",40.689236,-74.326223
6,270,Jim Harness,"9371 Harness Place, NW, Pickerington, OH, USA",39.912500,-82.763310
7,275,Whitney Ross,"22962 Carters Farm Ln, Middle Burg, VA, USA",39.912500,-82.763310
8,290,Shay Bar Nissim,"63 Parish Road, Needham, MA, USA",42.291992,-71.245252
9,321,Robert Gonshorowski,"322 Clifton Ave #202, Minneapolis, MN, USA",44.966023,-93.283392
